In [64]:
import json
from statistics import mean 

In [83]:
import requests
url = "http://localhost:8000/api/year_wise_playermatchstats/"
params = {'player_id':958,'start_date':'2020-01-07','year':2019}
response = requests.request("GET", url, headers=headers, params = params)

In [84]:
json_data = json.loads(response.text)
json_data = json_data.get('result')

In [90]:
# json_data

In [31]:
def return_int(d):
    try:
        d = int(d)
        return d
    except:
        return 0

In [88]:
def return_stats(json_data):     
    match_count = 0
    bowl_avg_list = []
    bat_inning_count = 0
    bowl_inning_count = 0
    bat_avg_list = []
    list_50 = []
    list_100 = []
    list_5w = []
    list_3w = []
    for m in json_data:
        scorecard = m.get('scorecard',[])
        match_count += 1
        bowling_avg = 0 
        batting_avg = 0
        for s in scorecard:
            bat = s.get('match_batting_inning')
            bowl = s.get('match_bowling_inning')
            
            bat = bat[0] if len(bat) != 0 else None
                
            if bat:
                
                order = bat.get('order')
                run = return_int(bat.get('runs'))
                f_w = bat.get('fall_of_wicket_over')
                how_out = bat.get('how_out')

                #calculate batting average
                if f_w != '' and how_out != '':
                    bat_avg_list.append(float(run))
                elif how_out != "not out" and how_out != 'retired hurt':
                    bat_avg_list.append(float(run))
                else:
                    bat_avg_list.append(0.0)
                #end

                #calculate 50
                list_50.append(1 if run >= 50 and run < 100 else 0)
                #end

                #calculate 100
                list_100.append(1 if run>=100 else 0)
                #end
                
                bat_inning_count+=1

            bowl = bowl[0] if len(bowl) != 0 else None

            if bowl:
                
                run_conceded = return_int(bowl.get('run_conceded',0))
                wickets = return_int(bowl.get('wickets',0))
                
                #calculate bowling average
                if wickets != 0:
                    bowling_avg = run_conceded/wickets
                    bowl_avg_list.append(bowling_avg)
                else:
                    bowl_avg_list.append(0.0)
                #end
                
                #5 and 3 w
                list_3w.append(1 if wickets >= 3 and wickets < 5 else 0)
                list_5w.append(1 if wickets >= 5 else 0)
                #end
                
                bowl_inning_count+=1

    print(match_count,bat_avg_list,bowl_avg_list,list_100,list_50,list_3w,list_5w,sep = '\n')
    bat_data = {
        'total_match':match_count,
        'inning':bat_inning_count,
        'bat_avg':mean(bat_avg_list) if len(bat_avg_list) != 0 else 0,
        '50': sum(list_50),
        '100':sum(list_100),
    }
    bowl_data = {
        'total_match':match_count,
        'inning': bowl_inning_count,
        'bowl_avg': mean(bowl_avg_list) if len(bowl_avg_list) != 0 else 0,
        '5w': sum(list_5w),
        '3w': sum(list_3w),
    }
    return bat_data , bowl_data

In [89]:
return_stats(json_data)

57
[85.0, 0.0, 4.0, 0.0, 19.0, 0.0, 136.0, 0.0, 12.0, 20.0, 0.0, 9.0, 0.0, 9.0, 51.0, 0.0, 120.0, 59.0, 28.0, 19.0, 1.0, 0.0, 26.0, 66.0, 72.0, 67.0, 77.0, 82.0, 18.0, 47.0, 18.0, 16.0, 25.0, 23.0, 13.0, 9.0, 100.0, 8.0, 67.0, 41.0, 84.0, 23.0, 3.0, 46.0, 6.0, 20.0, 7.0, 123.0, 116.0, 44.0, 0.0, 24.0, 60.0, 43.0, 45.0, 46.0, 104.0, 3.0, 23.0]
[]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[]
[]


({'total_match': 57,
  'inning': 59,
  'bat_avg': 36.728813559322035,
  '50': 15,
  '100': 7},
 {'total_match': 57, 'inning': 0, 'bowl_avg': 0, '5w': 0, '3w': 0})